# Hyperopt using sacred
The example function to optimize is:
$$x^2+y^2$$

Where x and y have added Gaussian noise with $\mu=0$ and $\sigma=1$.

This document shows quick examples of sequential and parallel optimization.

It is recommended to look at the example without sacred first as it has much better documentation.

In [1]:
import os
import math
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import hyperopt
from hyperopt import hp
from hyperopt.mongoexp import MongoTrials

# My imports
import objective

# Some parameters
n_trials = 150
local_avg_length = 15
n_workers = 5
jobs_per_worker = int(math.ceil(n_trials/n_workers))
db_name = "hopt-min-sacred"

In [2]:
search_space = dict(x=hp.uniform("x", -10, 10),
                    y=hp.uniform("y", -5, 20))

## Do a set of random sequential trials

In [3]:
random_trials_seq = hyperopt.Trials()
hyperopt.fmin(objective.sacred_hyperopt_objective,
              space=search_space,
              algo=hyperopt.rand.suggest,
              max_evals=n_trials,
              trials=random_trials_seq,
              verbose=2)

100%|██████████| 150/150 [00:49<00:00,  3.15it/s, best loss: 0.9888261762590211]


{'x': 0.3592081814638135, 'y': 0.6988205327789254}

## Do a set of parallel TPE trials

In [4]:
hopt_address = (f"mongo://{os.environ['MONGO_WRITE_IP']}:{os.environ['MONGO_PORT']}/" +
                db_name + "/jobs")
hopt_address

'mongo://localhost:27184/hopt-min-sacred/jobs'

In [5]:
# Set up the sub-processes
for i in range(n_workers):
    subprocess.Popen(f"bash hopt-workerN.sh {db_name} {jobs_per_worker}".split())

# Run the actual optimization (should finish super fast)
tpe_trials_par = MongoTrials(hopt_address, exp_key="tpe")
hyperopt.fmin(objective.sacred_hyperopt_objective,
              space=search_space,
              algo=hyperopt.tpe.suggest,
              max_evals=n_trials,
              trials=tpe_trials_par,
              verbose=2)

195it [03:20,  1.03s/it, best loss: 0.4454398376557718]                         


{'x': 1.6638083386117681, 'y': -0.7054565446930836}

## Now launch omniboard with the following command and you should see a lot of experiments

In [6]:
f"omniboard -m {os.environ['MONGO_WRITE_IP']}:{os.environ['MONGO_PORT']}:sacred"

'omniboard -m localhost:27184:sacred'